# Compare Stanford and Operator QC Criteria
Code author: Sahar H. El Abbadi
Date started: 2022-03-02
Date last edited: 2022-03-02

In [1]:
# Setup

# Imports
import pandas as pd
from data_manipulation_methods import load_clean_data, load_meter_data

# Load clean operator data
cm_1, cm_2, cm_3, ghg_1, ghg_2, kairos_1_ls23, kairos_1_ls25, kairos_2_ls23, kairos_2_ls25, kairos_3_ls23, kairos_3_ls25 = load_clean_data()

# Load meter data
cm_meter, ghg_meter, kairos_meter = load_meter_data()

In [15]:
# Combine data

# Method inputs
operator_report = cm_1
operator_meter = cm_meter

# Will begin method here
combined_df = operator_report.merge(operator_meter, on='PerformerExperimentID')

# Make column with easier name for coding for now.
combined_df.release_rate_kgh = combined_df['Last 60s (kg/h) - from Stanford']

# Philippine reports if we discard or not (discard = 1, keep = 0). Change this to have 1 if we keep, 0 if we discard
combined_df['stanford_kept'] =(1 - combined_df['QC: discard - from Stanford'])

# Make dataframe with all relevant info
operator_qc = pd.DataFrame()
operator_qc['overpass_id'] = combined_df.PerformerExperimentID
operator_qc['zero_release'] = combined_df.release_rate_kgh == 0
operator_qc['non_zero_release'] = combined_df.release_rate_kgh != 0  # True if we conducted a release
operator_qc['operator_kept'] = combined_df.stanford_kept
operator_qc['stanford_kept'] = combined_df.stanford_kept
operator_qc['operator_detected'] = combined_df.Detected
operator_qc['release_rate_kgh'] = combined_df.release_rate_kgh
operator_qc['operator_quantification'] = combined_df.FacilityEmissionRate

print(operator_qc)

     overpass_id  zero_release  non_zero_release  operator_kept  \
0            1.0         False              True              1   
1            2.0         False              True              1   
2            3.0         False              True              1   
3            4.0         False              True              1   
4            5.0          True             False              1   
..           ...           ...               ...            ...   
116        117.0         False              True              1   
117        118.0          True             False              0   
118        119.0         False              True              1   
119        120.0         False              True              1   
120        121.0         False              True              1   

     stanford_kept  operator_detected  release_rate_kgh  \
0                1               True         10.968637   
1                1              False          7.314810   
2                1

/var/folders/vp/7kbksk9n5bqdsc5w00b6k5zr0000gn/T/ipykernel_2822/1393490392.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  combined_df.release_rate_kgh = combined_df['Last 60s (kg/h) - from Stanford']
